In [162]:
import tulip as tlp
from tulip.interfaces import omega as omega_int
from tulip import transys, abstract, spec, synth
from visualization import graph_builder as gb
import networkx as nx
from tulip.transys import machines
from GR1_defaults import settings

from tulip import dumpsmach
import pickle

In [163]:
path = 'var_inc/'

In [164]:
# Variables
env_vars = {'env_bool':(0,1)}
sys_vars = {'num':(0,1)}

In [165]:
increment = False

In [166]:
# Initialization
env_init = {'env_bool=0'}
if increment:
    sys_init = {'num=0'}
else:
    sys_init = {'num=1'}

In [167]:
# Safety
env_safe = {'env_bool!=1'}
if increment:
    sys_safe = {'(num=0) -> (X(num)=num+1)',
                '(num=1) -> (X(num)=1)'}
else:
    sys_safe = {'(num=1) -> (X(num)=num-1)',
                '(num=0) -> (X(num)=0)'}



In [168]:
# Progress
env_prog = set()
if increment:
    sys_prog = {'num=1'}
else:
    sys_prog = {'num=0'}

In [169]:
specs = settings.set_specs(env_vars, sys_vars, env_init, sys_init,
                            env_safe, sys_safe, env_prog, sys_prog)

In [170]:
spec = tlp.synth._spec_plus_sys(specs, None, None, False, False)
# Automaton class found in omega/omega/symbolic/temporal.py
aut = omega_int._grspec_to_automaton(spec)

In [171]:
# Synthesizing system controller
ctrl = tlp.synth.synthesize(specs)
assert ctrl is not None, 'unrealizable'

removed 1 nodes from 3 total


In [172]:
 # Graphing
filename = path + "graph"
attributes = ['color', 'shape']

In [173]:
# Making a graph of the asynchronous GR(1) game with deadends.
g0 = gb.game_graph(aut, env='env', sys='sys', remove_deadends=False, qinit=aut.qinit)
h0 = gb._game_format_nx(g0, attributes)
pd0 = nx.drawing.nx_pydot.to_pydot(h0)
pd0.write_pdf(path + 'game.pdf')
with open(filename, "wb") as file:
    pickle.dump(g0, file)